# Maze Experiment Introduction

In this notebook, we will be running the MDP_Model on the Maze problem. This 2D simulation builds on a `gym-maze` package that can be found here: https://github.com/MattChanTK/gym-maze. Before beginning this simulation, please be sure to install the relevant packages on the github **Installation** section (pygame and numpy are also required)!

## Demonstration

First, a quick demonstration about what the game is. Essentially, there is a robot (circle) that starts on the blue start-point, then keeps taking steps (either with a designated policy or randomly), until it reaches the end point. Here is a simulation to demonstrate:

In [ ]:
# Load Libraries
import numpy as np
import pandas as pd
import math
import random
import gym
import gym_maze

In [ ]:
# Initializing the gym environment for a maze
env = gym.make("maze-sample-5x5-v0")

# Running the maze
observation = env.reset()
for _ in range(1000):
    
    env.render()
    action = env.action_space.sample() # your agent here (this takes random actions)
    observation, reward, done, info = env.step(action)
      
    if done:
      observation = env.reset()
env.close()

## Algorithm Goal

The goal on our end is to derive our dataset from the path that the robot takes. Every time it reaches a new coordinate, depending on whether the coordinate is the goal, the robot gets a "reward." With these datapoints, our algorithm should be able to learn an MDP to map out the optimal path through the maze!

The basic idea of the algorithm is that we start out with a small number of initial clusters, based solely on the `RISK` value available for each datapoint. Then, we attempt to find contradictions in the cluster transitions, with a contradiction defined as when points in the same cluster and take the same action actually go to different next clusters. During each iteration of the training, the model will split the cluster with the largest contradictions into two smaller clusters, and continue iterating until there are no more contradictions left or until the maximum number of iterations is reached. The result should be an MDP that represents the environment, with all its states and transitions.

## Dataset Creation

Loading relevant packages and functions - make sure to change the `sys.path.append` line with the relevant directory that contains the MDP Algorithm.

In [ ]:
# Set Working Directory
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

import sys
sys.path.append('/Users/janiceyang/Dropbox (MIT)/ORC UROP/Opioids/Algorithm/')
#sys.path.append('/Users/Amine/Dropbox (MIT)/Research/Opioids/Opioids Git/Algorithm/')

#from MDPtools import *
from model import MDP_model
from maze_functions import createSamples, opt_model_trajectory, opt_maze_trajectory, plot_paths
from testing import cluster_size, next_clusters, training_value_error, purity, plot_features

Now selecting parameters: here, we decide how many times we want the robot to run through the maze (`N`), when we want to start a new run if the robot takes too long (`T_max`), and the maze that we want the robot to run through (`mazes[x]`, with x being a number from the dictionary).

`reseed`' is set to `True` if we want the robot to travel to a different location within each cell every time it moves, while `False` ensures that the robot will start at a certain place in the initial coordinate, but move to the exact same place in the next coordinate. 

`r` is a float between 0 and 1 indicating the randomness percentage. For instance, if `r = 1`, the robot will take steps in completely random directions 100% of the time, while if `r = 0.5`, it will take half of its steps in the optimal direction (gotten by solving the maze MDP), but the other half randomly. 

In [ ]:
N = 50
T_max = 100
r = 0.4


# list of maze options to choose from:
mazes = {1: 'maze-v0',
         2: 'maze-sample-3x3-v0',
         3: 'maze-random-3x3-v0',
         4: 'maze-sample-5x5-v0',
         5: 'maze-random-5x5-v0',
         6: 'maze-sample-10x10-v0',
         7: 'maze-random-10x10-v0',
         8: 'maze-sample-100x100-v0',
         9: 'maze-random-100x100-v0',
         10: 'maze-random-10x10-plus-v0', # has portals 
         11: 'maze-random-20x20-plus-v0', # has portals 
         12: 'maze-random-30x30-plus-v0'} # has portals 

df = createSamples(N, T_max, mazes[2], r, reseed=True)
print(df)

The resulting `FEATURE_0` and `FEATURE_1` are the `x` and `y` coordinates respectively, while `ACTION` corresponds to the (`N`, `S`, `W`, `E`) directions. `RISK` is a reward of `1` if the endstate goal is reached, otherwise it is a negative factor of the maze size for all other locations. The robot does not change cells if it hits a wall, but if `reseed = True`, it can still change locations within the same cell. 

Here, we can visualize an example path that we are feeding as data into our algorithm. Note that it is not clear at all that there are grids or walls! The algorithm will train on these `N` paths to attempt to find the optimal policy. 

In [ ]:
plot_paths(df, 1)

## Running the Algorithm

Now, we can run the algorithm on the generated dataset! First, we can set some parameters, including `max_k` which is the number of clusters we want to end with, and thus determines the number of iterations during the splitting process. Since initial clustering is based solely on `RISK`, and there are only two groups (end state and all others), there will only be two initial clusters. The expected optimal `max_k` should be the maze size. 

`pfeatures` tells the algorithm how many features we have in the dataset, in this case 2. 

`classification` represents the type of classifier we want to use when splitting clusters with points that do not take the action where we found the contradiction. Options for this classifier include `'DecisionTreeClassifier'`, `'LogisticRegression'`, `'RandomForestClassifier'`, `'MLPClassifier'`, and `'AdaBoostClassifier'`. 

`split_classifier_params` passes in the arguments necessary to this split classifier, including items such as random_state or max_depth. `clustering` indicates the method used to form the initial clusters (based on RISK), with options of `'Agglomerative'`, `'KMeans'`, or `'Birch'`. `n_clusters` can be passed in to set the number of clusters to initialize. If `'Agglomerative'` is used, a `distance_threshold` must also be passed in to determine the distance between clusters (read `sklearn.cluster` documentation on `AgglomerativeClustering` for more details). 

`precision_thresh` determines the minimum decrease in value error necessary for the model to determine that a different split gives a better clustering. This value attempts to limit model complexity when improvements become only incremental.

`eta` is a constant factor that determines the incoherence threshold. Incoherence is defined as the the number of points in each cluster-action pair that do not go to the majority next cluster when a certain action is taken. During training, any clustering that results in a maximum incoherence above `eta*sqrt(n)/c`, where `n` is the number of datapoints given and `c` is the number of clusters at this current split, will be disregarded as too incoherent when finding the optimal clustering.

`th` is the threshold value that determines the minimum number of contradictory points required for a split of clusters.

`gamma` is the discount factor used when calculating value error, while `h` is the number of timesteps (horizon) on which to optimize the value error. When `h = -1`, we optimize over an infinite horizon.

`cv` is the number of folds for cross validation, only relevant when you run `model.fit_CV()`. 


In [ ]:
max_k = 9
pfeatures = 2

classification = 'DecisionTreeClassifier' 
split_classifier_params = {'random_state':0}
clustering = 'Agglomerative'
n_clusters = None
distance_threshold = 0.5
random_state = 0

precision_thresh = 1e-14
eta = 25
th = 0

gamma = 1
h = -1
cv = 5

Now, we can fit an algorithm model to this, using either `m.fit` or `m.fit_CV` (this one runs with `cv` rounds of cross validation, and takes the optimal split). `m.fit` with `optimize=True` will train the model on all the data, and retain the model with an optimal amount of splits.

In [ ]:
m = MDP_model()
m.fit(df, # df: dataframe in the format ['ID', 'TIME', ...features..., 'RISK', 'ACTION']
    pfeatures, # int: number of features
    h, # int: time horizon (# of actions we want to optimize)
    gamma, # discount factor
    max_k, # int: number of iterations
    distance_threshold, # clustering diameter for Agglomerative clustering
    cv, # number for cross validation
    th, # splitting threshold
    eta, # incoherence threshold
    precision_thresh, # precision threshold
    classification, # classification method
    split_classifier_params, # classification parameters
    clustering,# clustering method from Agglomerative, KMeans, and Birch
    n_clusters, # number of clusters for KMeans
    random_state,
    plot=True,
    optimize=False,
    verbose=False)

## Visualizing Results

If `plot = True` above, you should see that the final Value error drops to close to 0 when the number of clusters is 9, which is what should be the case for a 3 by 3 maze! Here is another visualization of how the correct clusters should be:

In [ ]:
plot_features(m.df_trained, 'FEATURE_0', 'FEATURE_1', c='OG_CLUSTER')

And here is a visualization of how the algorithm currently views where the clusters are:

In [ ]:
plot_features(m.df_trained, 'FEATURE_0', 'FEATURE_1', c='CLUSTER')

We can see that the clustering may not be completely perfect, and to quantify this, we can look at both the `training_error` and `purity`. The dataframe created by `training_error` tells us how the error was when we had that many clusters, so the error corresponding to `Clusters = 9` should be the lowest.

In [ ]:
m.training_error

The `purity` compares how much of the new clustering was made up of points from the same original cluster. We can see the percentage breakdown here - the higher the percentage, the better! 

In [ ]:
purity(m.df_trained)

## Comparing Solutions

We can see how well our model learned the maze by comparing the optimal path it found to the real solution. The real solution is found by getting the actual transition and reward matrices of the maze, then solving this MDP to get the optimal policy. This is what the correct path should be:

In [ ]:
opt_maze_trajectory(mazes[2])

And here, this is what our trained model found the solution to be (remember, our algorithm only got the messy data paths, and did not even know that there were 9 grid spaces!):

In [ ]:
opt_model_trajectory(m, mazes[2])

# Time for Bigger Mazes!

We can generate bigger mazes using the same method as above! Then, we will test to see if the optimal policy found by the maze is the same as the real optimal policy. 

#### Importing Modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

import sys
sys.path.append('/Users/janiceyang/Dropbox (MIT)/ORC UROP/Opioids/Algorithm/')

from model import MDP_model
from maze_functions import createSamples, opt_maze_trajectory, opt_model_trajectory, policy_accuracy, \
    get_maze_transition_reward, plot_paths
from testing import cluster_size, next_clusters, training_value_error, purity, plot_features, testing_value_error

mazes = {1: 'maze-v0',
         2: 'maze-sample-3x3-v0',
         3: 'maze-random-3x3-v0',
         4: 'maze-sample-5x5-v0',
         5: 'maze-random-5x5-v0',
         6: 'maze-sample-10x10-v0',
         7: 'maze-random-10x10-v0',
         8: 'maze-sample-100x100-v0',
         9: 'maze-random-100x100-v0',
         10: 'maze-random-10x10-plus-v0', # has portals 
         11: 'maze-random-20x20-plus-v0', # has portals 
         12: 'maze-random-30x30-plus-v0'} # has portals 

#### Creating Samples

In [ ]:
# Setting Parameters
N = 100
T_max = 25
r = 0.5
maze = mazes[4]

In [ ]:
df = createSamples(N, T_max, maze, r, reseed=True)
df

That's how the transition data looks like:

In [ ]:
plot_paths(df,1)

Checking how many points actually reach the end: 

In [ ]:
df.loc[df['ACTION']=='None']['ID'].count()

#### Fitting to Algorithm (CV Example)

This is an example of training with cross validation! For faster training, simply change `m.fit_CV` to `m.fit`. 

In [ ]:
# Setting parameters for model fitting
max_k = 20
classification = 'DecisionTreeClassifier'
split_classifier_params = {'random_state':0, 'max_depth':2}
clustering = 'Agglomerative'
n_clusters = None
distance_threshold = 0.5
random_state = 0
pfeatures = 2
h = -1
cv = 5
th = 0
eta = 25
precision_thresh = 1e-14

m = MDP_model()
m.fit_CV(df, # df: dataframe in the format ['ID', 'TIME', ...features..., 'RISK', 'ACTION']
    pfeatures, # int: number of features
    h, # int: time horizon (# of actions we want to optimize)
    gamma, # discount factor
    max_k, # int: number of iterations
    distance_threshold, # clustering diameter for Agglomerative clustering
    cv, # number for cross validation
    th, # splitting threshold
    eta, # incoherence threshold, calculated by eta*sqrt(datapoints)/clusters
    precision_thresh, # precision threshold
    classification, # classification method
    split_classifier_params, # classifier params
    clustering,# clustering method from Agglomerative, KMeans, and Birch
    n_clusters, # number of clusters for KMeans
    random_state,
    plot=True)

#### Observing Policies

First, let's see what the clustering that the model found actually looks like!

In [ ]:
plot_features(m.df_trained, 'FEATURE_0', 'FEATURE_1', 'CLUSTER')

And compare it with a clustering with the actual cells of the Maze.

In [ ]:
plot_features(m.df_trained, 'FEATURE_0', 'FEATURE_1', 'OG_CLUSTER')

Now, let's see what the optimal policy our model learns is:

In [ ]:
opt_model_trajectory(m, maze, 5, 0.7)

And here is an actual simulation of a point through the maze by taking the found optimal policy. Note that we have set a sink node as the bottom left corner, which is where the path will go once it has reached the goal state.

In [ ]:
f, r = get_maze_transition_reward(maze)
x0= np.random.rand(2)
m.opt_model_trajectory(x0, f)

Going through each point in the training data set, here is how many (by percentage) our optimal policy actually returned the correct action for. This is essentially the training accuracy:

In [ ]:
policy_accuracy(m, maze, m.df_trained)

Now, we will generate a new unseen test set using optimal parameters, and see how well the model does. This is the testing error:

In [ ]:
df_test = createSamples(50, T_max, maze, 0.3, reseed=True)
testing_value_error(df_test, m.df_trained, m.m, m.pfeatures, relative=False, h=-1)

Here is a metric that measures how good our classification model is in putting points in the right cluster:

In [ ]:
m.clus_pred_accuracy

And finally, here is the optimal policy for reference:

In [ ]:
opt_maze_trajectory(maze)

## Model Performance Based On Data Size

Now that we've seen how one model can perform based on one generated dataset, let's see how the model performs when given different numbers of paths to learn from. We will use a dataset with 200 paths.

In [ ]:
N = 200
T_max = 25
r = 0.5
maze = mazes[4]

df = createSamples(N, T_max, maze, r, reseed=True)
df

Now, we want to see how well the model trains on different subsets of the data, starting with when it only has access to the first 10 paths, `N=10`, all the way to all 200 paths, `N=200`. The subsets we look at can be seen in the list `Ns`. 

We will train a separate model for each data-size, then plot some trends in how the algorithm learns as training data increases.

In [ ]:
# Setting parameters for model fitting
max_k = 25
classification = 'DecisionTreeClassifier'
split_classifier_params = {'random_state':0, 'max_depth':2}
clustering = 'Agglomerative'
n_clusters = None
distance_threshold = 0.5
random_state = 0
pfeatures = 2
gamma = 1
actions = [0, 1, 2, 3]
h = -1
cv = 5
th = 0
eta = 25
precision_thresh = 1e-14

Ns = [10, 20, 30, 40, 50, 70, 90, 110, 130, 150, 170, 200]
df_full = df.copy()

models=[]
    
# Training models 
for n in Ns:
    df_small = df_full.loc[df_full['ID']<n]
    
    m = MDP_model()
    m.fit(df_small, # df: dataframe in the format ['ID', 'TIME', ...features..., 'RISK', 'ACTION']
        pfeatures, # int: number of features
        h, # int: time horizon (# of actions we want to optimize)
        gamma, # discount factor
        max_k, # int: number of iterations
        distance_threshold, # clustering diameter for Agglomerative clustering
        cv, # number for cross validation
        th, # splitting threshold
        eta, # incoherence threshold
        precision_thresh, # precision threshold
        classification, # classification method
        split_classifier_params, # classification params
        clustering,# clustering method from Agglomerative, KMeans, and Birch
        n_clusters, # number of clusters for KMeans
        random_state,
        plot=False,
        optimize=True)
    print('N=', n, ' completed')
    models.append(m)

### Errors

Now, we are going to evaluate how the various models have performed in terms of their in-sample training error (at the point where the model has reached the optimal clustering and stopped training), as well as the out-of-sample testing error based on a newly generated dataset with the same parameters.

In [ ]:
from testing import testing_value_error

N = 200
T_max = 25
r = 0.4
maze = mazes[4]
# Creating a test set with same parameters as training set
df_test = createSamples(N, T_max, maze, r, reseed=True)

# In & out sample training and testing value errors: 
training_errors = []
testing_errors = []
for m in models: 
    tr_err = m.training_error.loc[m.training_error['Clusters']==m.opt_k]['Error'].min()
    te_err = testing_value_error(df_test, m.df_trained, m.m, m.pfeatures, gamma, relative=False, h=-1)
    training_errors.append(tr_err)
    testing_errors.append(te_err)

fig1, ax1 = plt.subplots()
ax1.plot(Ns, training_errors, label='Training Error')
ax1.plot(Ns, testing_errors, label='Testing Error')
ax1.set_title('Testing and Training Errors by N')
ax1.set_xlabel('N training data size')
ax1.set_ylabel('Error')
ax1.legend()

### Accuracies

We also want to measure both training and testing accuracies, measured as how well the model learns and maps each point to the correct original clustering of the maze, and how these values change as data size increases. 

In [ ]:
from testing import generalization_accuracy

tr_acc, test_acc = generalization_accuracy(models, df_test, Ns)

### Optimality Gap

Next, we want to measure the optimality gap, where given a point that begins randomly in the starting cell, we want to measure the value difference after `t_max` steps between two scenarios: 1) if this point takes only the optimal action through the maze, and 2) if this point takes the action that the model prescribes based on the model's solved MDP. In both scenarios, the point progresses through the maze according to the actual maze dynamics; the only difference lies in what sequence of actions it is given.


We will once again plot the change in this value as data size increases.

In [ ]:
from maze_functions import get_maze_MDP, get_maze_transition_reward, value_diff

# Set Parameters
P, R = get_maze_MDP(maze)
K = 100
f, rw = get_maze_transition_reward(maze)

opt_gap = value_diff(models, Ns, K, T_max, P, R, f, rw)
fig1, ax1 = plt.subplots()
ax1.plot(Ns, opt_gap)
ax1.set_title('Optimality Gap by Data Size N')
ax1.set_xlabel('N training data size')
ax1.set_ylabel('|V_alg-V*|')